In [1]:
!git clone https://github.com/michaelwwk/appStoreAnalytics.git

Cloning into 'appStoreAnalytics'...
remote: Enumerating objects: 3374, done.
remote: Counting objects: 100% (424/424), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 3374 (delta 270), reused 419 (delta 268), pack-reused 2950
Receiving objects: 100% (3374/3374), 1.72 MiB | 5.09 MiB/s, done.
Resolving deltas: 100% (2243/2243), done.


In [2]:
import pandas as pd
import numpy as np
import json
from google.cloud import bigquery, storage
!pip install pyspark
from pyspark.sql import SparkSession
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk
nltk.download('punkt')
import warnings
warnings.filterwarnings("ignore")
import sklearn.preprocessing as pp
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=766c95c77ad12fd10ef033a8a78be6f7780b0befa47cdca85175d3cb49c0e1b6
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Start Spark session
spark = SparkSession.builder.master("local").appName("appStoreAnalytics").config('spark.ui.port', '4050').getOrCreate()

In [6]:
# googleAPI_json_path = "/content/drive/MyDrive/Colab Notebooks/big-data-analytics-415801-f8364cc7c7d5.json"
# project_id = "big-data-analytics-415801"
# client = bigquery.Client.from_service_account_json(googleAPI_json_path, project = project_id)

In [7]:
# Define your BigQuery project ID and table ID
project_id = "big-data-analytics-415801"
table_id = 'dev_cleanData.cleanGoogleMain'

# SQL query to fetch the data from the table
sql_query = f"SELECT * FROM {table_id} limit 5000"

# Read data from BigQuery into a DataFrame
googleMain = pd.read_gbq(sql_query, project_id=project_id)

In [8]:
googleMain.head()

title  \
0                       CWD Smart   
1        CCSWE App Manager (Root)   
2  Click Counter and Tally Counte   
3                 Developer Tools   
4  EverTranslator ScreenTranslate   

                                         description  \
0  Thanks for choosing CWD Smart app.\r\nYou can ...   
1  CCSWE App Manager</b> was designed for users w...   
2  Card Counter is a powerful and easy to use Tal...   
3  Developer tools</b> is an application for andr...   
4  Translate text anytime and everywhere, even yo...   

                                             summary  installs minInstalls  \
0  CWD Smart can help you to control your Colling...      500+         500   
1  CCSWE App Manager is the ultimate package disa...    1,000+        1000   
2  Card Counter is a tally counter that makes cou...    1,000+        1000   
3  Developer tools app is a powerful application ...   10,000+       10000   
4  Screen translator - Translate text anytime and...  100,000+      100000   

  realInstalls      score ratings reviews price   free currency  offersIAP  \
0          628        2.6       5       0   0.0   True      USD      False   
1         1011       3.75      25       6  2.49  False      USD      False   
2         4556        3.6      23       3   0.0   True      USD       True   
3        35012  4.2083335     187      20   0.0   True      USD      False   
4       494071       3.41    2376     103   0.0   True      USD      False   

  inAppProductPrice             developer  genre genreId contentRating  \
0              None  Collingwood Lighting  Tools   TOOLS      Everyone   
1              None         Cory Charlton  Tools   TOOLS      Everyone   
2    $0.99 per item                   AGF  Tools   TOOLS      Everyone   
3              None         Roxanne Crete  Tools   TOOLS      Everyone   
4              None            Firemaples  Tools   TOOLS      Everyone   

  contentRatingDescription adSupported      released lastUpdatedOn version  \
0                     None       False          None  May 19, 2022  2.24.0   
1                     None       False   Oct 9, 2020  Sep 24, 2023   6.6.1   
2                     None        True  Jul 30, 2018  Sep 19, 2021     1.7   
3                     None        True  Apr 24, 2019  Aug 24, 2023     8.0   
4                     None        True  Dec 29, 2016  Jan 20, 2024  3.1.25   

                                appId categories_list min_inAppProductPrice  \
0           com.collingwood.smarthome         [Tools]                  None   
1  com.ccswe.appmanager.root_20201007         [Tools]                  None   
2                    com.card.counter         [Tools]                 0.99    
3                 rox.developer.tools         [Tools]                  None   
4           tw.firemaples.onscreenocr         [Tools]                  None   

  max_inAppProductPrice  1starrating_no  2starrating_no  3starrating_no  \
0                  None               3               0               0   
1                  None               6               0               0   
2                  None               4               0               4   
3                  None              19               9               7   
4                  None             641             131             247   

   4starrating_no  5starrating_no  
0               0               2  
1               6              12  
2               4               9  
3              25             124  
4             247            1064

In [9]:
googleMain.to_csv("/content/drive/MyDrive/Colab Notebooks/googleMain.csv")

# Read CSV files into PySpark DataFrame
googleMain_sparkDF = spark.read.format("csv") \
                    .option("inferSchema", "true") \
                    .option("header", "true") \
                    .option("multiline", "true") \
                    .option("escape", "\"") \
                    .csv("/content/drive/MyDrive/Colab Notebooks/googleMain.csv")

## Data preparation

In [11]:
filter_list = []

def preprocess_text(text):
    # Tokenize the input text
    tokens = word_tokenize(text)

    # Convert all characters to lower case
    tokens = [t.lower() for t in tokens]

    # Remove Stopwords and filtered words
    stop_words = set(stopwords.words('english'))
    tokens = [t for t in tokens if t not in stop_words and t not in filter_list]

    # Remove Punctuations
    tokens = [t for t in tokens if t not in string.punctuation]

    # Remove Numbers/Numerics
    tokens = [t for t in tokens if not t.isnumeric()]

    # Lemmatize
    lemmatizer = nltk.WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(w) for w in tokens]

    return tokens

In [12]:
text = googleMain['title'] + ' ' + googleMain['description'] + ' ' + googleMain['summary']

In [13]:
tfidf = TfidfVectorizer(tokenizer = preprocess_text, min_df = 2, max_df = 0.7)
tdm = tfidf.fit_transform(text)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [14]:
tdm

<5000x15406 sparse matrix of type '<class 'numpy.float64'>'
	with 339438 stored elements in Compressed Sparse Row format>

In [15]:
n_components = 50  # This no. must be the same or smaller than the no. of rows!! Typically set between 100 to 300
svd = TruncatedSVD(n_components=n_components)
tfidf_reduced = svd.fit_transform(tdm)
tfidf_df = pd.DataFrame(tfidf_reduced, columns=[f'component_{i}' for i in range(n_components)])

In [16]:
googleMain.columns

Index(['title', 'description', 'summary', 'installs', 'minInstalls',
       'realInstalls', 'score', 'ratings', 'reviews', 'price', 'free',
       'currency', 'offersIAP', 'inAppProductPrice', 'developer', 'genre',
       'genreId', 'contentRating', 'contentRatingDescription', 'adSupported',
       'released', 'lastUpdatedOn', 'version', 'appId', 'categories_list',
       'min_inAppProductPrice', 'max_inAppProductPrice', '1starrating_no',
       '2starrating_no', '3starrating_no', '4starrating_no', '5starrating_no'],
      dtype='object')

In [17]:
googleMainFiltered= googleMain[['realInstalls', 'score', 'reviews', 'price', 'free', 'genre', 'contentRating', 'adSupported', '1starrating_no', '2starrating_no', '3starrating_no', '4starrating_no', '5starrating_no']].loc[googleMain.title != 'None']
googleMainFiltered.head()

realInstalls      score reviews price   free  genre contentRating  \
0          628        2.6       0   0.0   True  Tools      Everyone   
1         1011       3.75       6  2.49  False  Tools      Everyone   
2         4556        3.6       3   0.0   True  Tools      Everyone   
3        35012  4.2083335      20   0.0   True  Tools      Everyone   
4       494071       3.41     103   0.0   True  Tools      Everyone   

  adSupported  1starrating_no  2starrating_no  3starrating_no  4starrating_no  \
0       False               3               0               0               0   
1       False               6               0               0               6   
2        True               4               0               4               4   
3        True              19               9               7              25   
4        True             641             131             247             247   

   5starrating_no  
0               2  
1              12  
2               9  
3             124  
4            1064

In [18]:
# create dummy variables (one hot)
googleMainFiltered['free'] = googleMainFiltered['free'].map({'False':0, 'True':1}).astype('int64')
googleMainFiltered['adSupported'] = googleMainFiltered['adSupported'].map({'False':0, 'True':1}).astype('int64')
googleMainFiltered = pd.concat([googleMainFiltered, pd.get_dummies(googleMainFiltered['genre'], prefix='genre')], axis=1)
googleMainFiltered = pd.concat([googleMainFiltered, pd.get_dummies(googleMainFiltered['contentRating'], prefix='contentRating')], axis=1)
googleMainFiltered.pop('genre')
googleMainFiltered.pop('contentRating')

0           Everyone
1           Everyone
2           Everyone
3           Everyone
4           Everyone
5           Everyone
6               Teen
7           Everyone
8         Mature 17+
9           Everyone
10          Everyone
11          Everyone
12          Everyone
13          Everyone
14          Everyone
15          Everyone
16          Everyone
17          Everyone
18          Everyone
19          Everyone
20          Everyone
21          Everyone
22          Everyone
23          Everyone
24          Everyone
25          Everyone
26          Everyone
27          Everyone
28          Everyone
29          Everyone
30          Everyone
31          Everyone
32          Everyone
33          Everyone
34          Everyone
35          Everyone
36          Everyone
37          Everyone
38              Teen
39          Everyone
40          Everyone
41          Everyone
42          Everyone
43          Everyone
44          Everyone
45              Teen
46          Everyone
47          E

In [19]:
# googleMainFiltered = googleMainFiltered.copy() # option 1 ('realInstalls', 'score', 'reviews', 'price', 'free', 'genre', 'contentRating', 'containsAds')
# googleMainFiltered = tfidf_df # option 2 ('title', 'description', 'summary')
googleMainFiltered = pd.concat([googleMainFiltered, tfidf_df], axis = 1) # option 3 ('realInstalls', 'score', 'reviews', 'price', 'free', 'genre', 'contentRating', 'containsAds' + 'title', 'description', 'summary')

googleMainFiltered.replace('None', np.nan, inplace=True)
googleMainFiltered.replace('nan', np.nan, inplace=True)
googleMainFiltered = googleMainFiltered.dropna()
googleMainFiltered = googleMainFiltered.astype('float64')

In [20]:
googleMainFiltered.shape

(5000, 113)

In [21]:
# import pandas as pd
# from statsmodels.stats.outliers_influence import variance_inflation_factor

# while True:
#     vif = pd.DataFrame()
#     vif["Features"] = googleMainFiltered.columns
#     vif["VIF"] = [variance_inflation_factor(googleMainFiltered.values, i) for i in range(googleMainFiltered.shape[1])]

#     max_vif = vif["VIF"].max()

#     if max_vif > 10:
#         column_to_remove = vif.loc[vif["VIF"].idxmax(), "Features"]
#         googleMainFiltered = googleMainFiltered.drop(column_to_remove, axis=1)
#     else:
#         break

#### K-Nearest Neighbours

In [22]:
# to display the results
def knn_showResults(qind, dis, ind, df_condensed, df):
  for i, d in zip(ind[qind], dis[qind]) :
    #retrieve the original row index, to access 'data'
    reali = df_condensed.loc[i, 'index']
    print('=====')
    print('\tRow Index: ' + str(reali) + '\tDistance: ' + str(d) + '\tApp Title: ' + df.loc[reali, 'title'] )
    #access the values in 'df_condensed' using new row index
    # print(str(df_condensed.loc[i]))
    print(df.loc[reali, 'title'] + "\t"+ df.loc[reali, 'summary'])

#### Cosine-Similarity

In [23]:
# find top k movies most similar to q
def cosine_showResults(q, k, df, tdm):
    print(df['title'][q])
    vals = cosine_similarity(tdm[q], tdm)
    idx_asc = vals.argsort()[0][-k:] #get index of highest sim values
    idx_dsc = idx_asc[::-1] #reverse the indices
    print(idx_dsc)
    flatv = np.sort(vals[0])
    vk_asc = flatv[-k:]
    vk_dsc = vk_asc[::-1] #reverse the list
    if(vk_dsc[1]==0):
        print("No similar movie is found")
    else:
        for v, i in zip(vk_dsc, idx_dsc):
          # print(df['title'][i]+'\t'+str(v)+"\t"+str(i)+"\t"+str(df['index'][i])+'\t'+str(df['genre'][i]))
          print('=====')
          print(df['title'][i]+'\t'+str(v))
          print('\t'+df['summary'][i])

#### Embedding method (Doc2Vec model)

In [ ]:
text = googleMain_sparkDF['title'] + ' ' + googleMain_sparkDF['description'] + ' ' + googleMain_sparkDF['summary']

In [57]:
text_tokens = [word_tokenize(t.lower()) for t in text]
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(text_tokens)]
tagged_data[:3]

model = Doc2Vec(vector_size=64, min_count=2, epochs=40)
model.build_vocab(tagged_data)

# Train the model using our data
model.train(tagged_data, total_examples=model.corpus_count, epochs=40)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Run model results

In [29]:
scaler = pp.MinMaxScaler()
googleMainFilteredScaled = scaler.fit_transform(googleMainFiltered)

In [30]:
googleMainFiltered.reset_index(inplace=True)
googleMain.reset_index(inplace=True)

In [31]:
nbrs_scaled = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(googleMainFilteredScaled)
dis_scaled, ind_scaled = nbrs_scaled.kneighbors(googleMainFilteredScaled)

#### K-Nearest Neighbours

In [41]:
knn_showResults(1010, dis_scaled, ind_scaled, googleMainFiltered, googleMain) # 50 # 52

=====
	Row Index: 1010	Distance: 0.0	App Title: بحر حرب - لعبة ألغاز الحروف
بحر حرب - لعبة ألغاز الحروف	A puzzle game that challenges your intelligence. Try to find two words of the same letter, but in a different order
=====
	Row Index: 684	Distance: 0.42411097799327396	App Title: Hangman Game of Thrones Style
Hangman Game of Thrones Style	The classic Hangman game with adults only, Game of Thrones twist
=====
	Row Index: 3699	Distance: 0.6272542705264087	App Title: Unlucky Boy Rescue 3
Unlucky Boy Rescue 3	Try to solve funny puzzles at every level in this cool point and click game
=====
	Row Index: 327	Distance: 0.6480898603254653	App Title: Holmes Cryptic Cipher Puzzle
Holmes Cryptic Cipher Puzzle	Word game - swap letters, crack the code and reveal a quote from Sherlock Holmes
=====
	Row Index: 3314	Distance: 0.6491179533056225	App Title: Minesweeper Classic
Minesweeper Classic	Enjoy Minesweeper! Learn to play the classic logic game from the 90's.
=====
	Row Index: 3515	Distance: 0.6

#### Cosine-Similarity

In [43]:
cosine_showResults(1010, 6, googleMain, tdm) # tdm only

بحر حرب - لعبة ألغاز الحروف
[1010 4759  748 4762 1619 1102]
=====
بحر حرب - لعبة ألغاز الحروف	1.0000000000000002
	A puzzle game that challenges your intelligence. Try to find two words of the same letter, but in a different order
=====
Kelime Şöleni: Kelime Oyunu	0.4064324000162741
	Play word puzzles and word games! Brain games and the most fun games.
=====
Freek Words - Word Connect Puz	0.3807621256078448
	Do you enjoy word puzzle games? Here comes the perfect word games for you!
=====
Wordlook - Guess The Word Game	0.37330052261822766
	GUESS THE WORD WITH COLORED TILES! Word Games in Wordlook Word Puzzles!
=====
Word Guru: 5 in 1 Search Word 	0.3626638064181484
	Solve word challenges & test vocabulary in 5 fun and addictive word puzzle games
=====
Word Porn Word Puzzles Game Mo	0.3555846220987986
	World's Most Addictive Word Puzzle Game. Download it for free now.


#### Embedding method (Doc2Vec model)

In [84]:
newData = """
This application helps you to securely store your data in a vault with a pin code. it can include photos, videos, messages, and also your own notes such as passwords.
"""

test_doc = word_tokenize(newData.lower())
test_vec = model.infer_vector(test_doc)
results = model.docvecs.most_similar(positive=[test_vec],topn=5)

#check the results. Do they make sense?
for i, d in results:
  print(f"Score:\n{d}\n")
  print(f"Title:\n{googleMain['title'][i]}\n")
  print(f"Details:\n{text[i]}")
  print("-" * 50)

Score:
0.6059494018554688

Title:
Recover Deleted All Files, Pho

Details:
Recover Deleted All Files, Pho Recover Deleted All Files, Photos and Contacts

Photo & Video Restore or Photo Video Recovery app with Contacts Recovery or Restore Contacts feature is an awesome Recovery app for deleted Photo and Video. This application will help you to restore all the deleted photos, videos , files and contacts with this simple android free app!

Data Recovery Deleted Photos, Video, Files Restore is the newly developed application that helps the user to recover lost data if they got deleted intentional or unintentionally. 

All user has to scan whether the whole device or just a selected and wait for few seconds and the system will automatically search the location and retrieve all items of lost data and it’s up to user now whether to store the data retrieved or permanently delete the data so no other can find the data on your device.

Contact Recovery can restore your deleted contacts from your

## Chosen method:
#### Embedding method (Doc2Vec model)

In [ ]:
# Prep data to import into GBQ

In [ ]:
# Stop Spark session
# spark.stop()